In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도


# 옵션 추가
#pd.set_option('display.max_rows', None)

In [3]:
url = '/content/drive/MyDrive/cleaned_Data.csv'
df = pd.read_csv(url, encoding = 'utf-8')

In [4]:
df.head()

,상호명,업종중분류명,업종소분류명,표준산업분류명,행정동명,위도,경도,합친데이터,네이버키워드,가게_URL,가게_주소,가게_평점,평점에_참여한_인원,방문자_리뷰,리뷰_총인원
0,카페65도씨,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.469868,126.812189,커피점/카페커피전문점/카페/다방비알콜 음료점업,범박동%20카페65도씨,https://m.place.naver.com/restaurant/37459407,경기 부천시 양지남로 39,4.31,252,/분위기좋고 대화하기 좋아요/매장은 넓어 보여 분위기 좋을것같아 방문 청결상태 별로...,101
1,블럭앤카페,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.472723,126.807531,커피점/카페커피전문점/카페/다방비알콜 음료점업,범박동%20블럭앤카페,https://m.place.naver.com/restaurant/1262904745,경기 부천시 범안로96번길 23 4단지상가 3층,0.00,0,/좋아요/가까워서 아이의 놀이터^^/사장님 좋으세요~/신나는슬라임 비즈랑 블럭놀이 ...,5
2,한우세계,한식,갈비/삼겹살,한식 음식점업,범박동,37.470159,126.807956,한식갈비/삼겹살한식 음식점업,범박동%20한우세계,https://m.place.naver.com/restaurant/31629816,경기 부천시 범안로 130-21,0.00,0,/좋아요/좋아요/좋아요/국/고기가 맛있어요/좋아요/굿/굿/좋아요,72
3,마론,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.473363,126.812925,커피점/카페커피전문점/카페/다방비알콜 음료점업,범박동%20마론,https://m.place.naver.com/restaurant/37435543,경기 부천시 양지로 115 1층 카페마론,4.40,1438,"/가격도 저렴하고 빵 맛있어요. 케익생크림도 안느끼하고 안달고 맛나고, 시나몬롤도 ...",847
4,부어치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,범박동,37.468826,126.806270,닭/오리요리후라이드/양념치킨치킨 전문점,범박동%20부어치킨,https://m.place.naver.com/restaurant/1654229456,경기 부천시 범안로219번길 19,4.69,92,/제로웨이스트 실천해서 통을 가지고 가는데 감사하게도 흔쾌히 괜찮다고 해주셔서 전화...,80


In [5]:
df = df.drop(['합친데이터'], axis =1)

In [6]:
df

,상호명,업종중분류명,업종소분류명,표준산업분류명,행정동명,위도,경도,네이버키워드,가게_URL,가게_주소,가게_평점,평점에_참여한_인원,방문자_리뷰,리뷰_총인원
0,카페65도씨,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.469868,126.812189,범박동%20카페65도씨,https://m.place.naver.com/restaurant/37459407,경기 부천시 양지남로 39,4.31,252,/분위기좋고 대화하기 좋아요/매장은 넓어 보여 분위기 좋을것같아 방문 청결상태 별로...,101
1,블럭앤카페,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.472723,126.807531,범박동%20블럭앤카페,https://m.place.naver.com/restaurant/1262904745,경기 부천시 범안로96번길 23 4단지상가 3층,0.00,0,/좋아요/가까워서 아이의 놀이터^^/사장님 좋으세요~/신나는슬라임 비즈랑 블럭놀이 ...,5
2,한우세계,한식,갈비/삼겹살,한식 음식점업,범박동,37.470159,126.807956,범박동%20한우세계,https://m.place.naver.com/restaurant/31629816,경기 부천시 범안로 130-21,0.00,0,/좋아요/좋아요/좋아요/국/고기가 맛있어요/좋아요/굿/굿/좋아요,72
3,마론,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,범박동,37.473363,126.812925,범박동%20마론,https://m.place.naver.com/restaurant/37435543,경기 부천시 양지로 115 1층 카페마론,4.40,1438,"/가격도 저렴하고 빵 맛있어요. 케익생크림도 안느끼하고 안달고 맛나고, 시나몬롤도 ...",847
4,부어치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,범박동,37.468826,126.806270,범박동%20부어치킨,https://m.place.naver.com/restaurant/1654229456,경기 부천시 범안로219번길 19,4.69,92,/제로웨이스트 실천해서 통을 가지고 가는데 감사하게도 흔쾌히 괜찮다고 해주셔서 전화...,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,영구스피자,패스트푸드,피자전문,"피자, 햄버거, 샌드위치 및 유사 음식점업",괴안동,37.476900,126.808064,괴안동%20영구스피자,https://m.place.naver.com/restaurant/38345043,경기 부천시 범안로 55 홈타운프라자 104호 영구스피자,4.40,60,"/맛있어요/피자 굿/영구스피자 굿/굿/가성비 좋고 , 맛도 괜찮네요 ~/좋아요/맛있...",47
274,바지락손칼국수,분식,라면김밥분식,분식 및 김밥 전문점,괴안동,37.478075,126.807570,괴안동%20바지락손칼국수,https://m.place.naver.com/restaurant/37592268,경기 부천시 소사동로 119,4.20,173,/새악 팥죽 맛있게 먹었어요/바지락신선도가별로인지 국물이 별로ㅠ 팥 칼국수는 맛있어...,102
275,신정상회,한식,한식/백반/한정식,한식 음식점업,괴안동,37.482803,126.809101,괴안동%20신정상회,https://m.place.naver.com/restaurant/1089637756,경기 부천시 소삼로 11,4.22,26,/굿/ㅅ/친절해요/좋아요/좋아요/좋아요/좋아요/좋아요/친절하십니다^^/TV에도 나온...,11
276,네네치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,괴안동,37.474207,126.804636,괴안동%20네네치킨,https://m.place.naver.com/restaurant/11779297,경기 부천시 소사동로 93,4.39,91,/./맛나요!/네네치킨은 파닭이 진리인데 아이랑 같이 먹으려고 후라이드로 주문했어요...,50


In [7]:
df.columns

Index(['상호명', '업종중분류명', '업종소분류명', '표준산업분류명', '행정동명', '위도', '경도', '네이버키워드',
       '가게_URL', '가게_주소', '가게_평점', '평점에_참여한_인원', '방문자_리뷰', '리뷰_총인원'],
      dtype='object')

In [8]:
df['분류명합산'] = df['업종중분류명'] + df['업종소분류명'] + df['표준산업분류명']

In [9]:
df['분류명합산']

0             커피점/카페커피전문점/카페/다방비알콜 음료점업
1             커피점/카페커피전문점/카페/다방비알콜 음료점업
2                       한식갈비/삼겹살한식 음식점업
3             커피점/카페커피전문점/카페/다방비알콜 음료점업
4                 닭/오리요리후라이드/양념치킨치킨 전문점
                     ...               
273    패스트푸드피자전문피자, 햄버거, 샌드위치 및 유사 음식점업
274                 분식라면김밥분식분식 및 김밥 전문점
275                  한식한식/백반/한정식한식 음식점업
276               닭/오리요리후라이드/양념치킨치킨 전문점
277                 분식라면김밥분식분식 및 김밥 전문점
Name: 분류명합산, Length: 278, dtype: object

In [10]:
df['분류명합산'] = df['분류명합산'].str.replace("/", " ")
df['분류명합산']

0             커피점 카페커피전문점 카페 다방비알콜 음료점업
1             커피점 카페커피전문점 카페 다방비알콜 음료점업
2                       한식갈비 삼겹살한식 음식점업
3             커피점 카페커피전문점 카페 다방비알콜 음료점업
4                 닭 오리요리후라이드 양념치킨치킨 전문점
                     ...               
273    패스트푸드피자전문피자, 햄버거, 샌드위치 및 유사 음식점업
274                 분식라면김밥분식분식 및 김밥 전문점
275                  한식한식 백반 한정식한식 음식점업
276               닭 오리요리후라이드 양념치킨치킨 전문점
277                 분식라면김밥분식분식 및 김밥 전문점
Name: 분류명합산, Length: 278, dtype: object

In [11]:
df['분류명합산'] = df['분류명합산'].str.replace(",", "")
df['분류명합산']

0           커피점 카페커피전문점 카페 다방비알콜 음료점업
1           커피점 카페커피전문점 카페 다방비알콜 음료점업
2                     한식갈비 삼겹살한식 음식점업
3           커피점 카페커피전문점 카페 다방비알콜 음료점업
4               닭 오리요리후라이드 양념치킨치킨 전문점
                    ...              
273    패스트푸드피자전문피자 햄버거 샌드위치 및 유사 음식점업
274               분식라면김밥분식분식 및 김밥 전문점
275                한식한식 백반 한정식한식 음식점업
276             닭 오리요리후라이드 양념치킨치킨 전문점
277               분식라면김밥분식분식 및 김밥 전문점
Name: 분류명합산, Length: 278, dtype: object

In [12]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['분류명합산']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

#### 크롤링으로 텍스트 리뷰를 모은 컬럼 또한 같은 작업을 진행합니다.


In [13]:
# 리뷰 텍스트 데이터 간의 텍스트 피쳐 벡터라이징
count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['방문자_리뷰']) 

# 리뷰 텍스트 간의 코사인 유사도 따지기
place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]


####Step 5. 다양한 요소를 종합하는 계산식을 구성하고 추천 받는 함수를 짠다<br/>
이제 데이터를 모두 가져오고, 후 가공까지 마쳤다면 추천 알고리즘을 짜봅시다.<br/>
공식 1 : 카테고리가 얼마나 유사한지<br/>
공식 2 : 블로그 리뷰가 얼마나 유사한지<br/>
공식 3 : 블로그 리뷰가 얼마나 많이 올라왔는지<br/>
공식 4 : 블로그 별점이 얼마나 높은지<br/>
공식 5 : 블로그 별점 평가가 얼마나 많이 됐는지<br/>
위 공식 5개를 적절한 비율로 가중치를 주면서, 체감상 더 추천이 잘 되는 최적의 공식을 짜야합니다.


In [14]:
# 공식 1~5의 중요성을 짬뽕시키는 공식
# * 0.003 등의 가중치를 줘서 조절합니다.

place_simi_co = (
                 + place_simi_cate * 0.9 # 공식 1. 카테고리 유사도
                 + place_simi_review * 0.001 # 공식 2. 리뷰 텍스트 유사도
                 + np.repeat([df['평점에_참여한_인원'].values], len(df['평점에_참여한_인원']) , axis=0) * 0.001  # 공식 3. 블로그 리뷰가 얼마나 많이 올라왔는지
                 + np.repeat([df['가게_평점'].values], len(df['가게_평점']) , axis=0) * 0.005            # 공식 4. 블로그 별점이 얼마나 높은지
                 + np.repeat([df['리뷰_총인원'].values], len(df['리뷰_총인원']) , axis=0) * 0.001    # 공식 5. 블로그 별점 평가가 얼마나 많이 됐는지
                 )


# 아래 place_simi_co_sorted_ind 는 그냥 바로 사용하면 됩니다.
place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 


# 최종 구현 함수
def find_simi_place(df, sorted_ind, place_name, top_n=5):
    
    place_title = df[df['상호명'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

In [20]:
place_index = place_title.index.values
place_index

array([113, 135, 180, 247])

In [17]:
 place_title = df[df['상호명'] == '파리바게뜨']
 place_title

,상호명,업종중분류명,업종소분류명,표준산업분류명,행정동명,위도,경도,네이버키워드,가게_URL,가게_주소,가게_평점,평점에_참여한_인원,방문자_리뷰,리뷰_총인원,분류명합산
113,파리바게뜨,제과제빵떡케익,제과점,제과점업,소사본동,37.481152,126.792857,소사본동%20파리바게뜨,https://m.place.naver.com/restaurant/19258625,경기 부천시 소사동로 82 센트럴프라자 105호,4.34,599,"/샐러드 가성비가 좋아요.. 케이준샐러드 고기도 꽉꽉, 채소도 꽉꽉 담겨있어요 치킨...",427,제과제빵떡케익제과점제과점업
135,파리바게뜨,제과제빵떡케익,제과점,제과점업,소사본동,37.481974,126.793149,소사본동%20파리바게뜨,https://m.place.naver.com/restaurant/19258625,경기 부천시 소사동로 82 센트럴프라자 105호,4.34,599,"/샐러드 가성비가 좋아요.. 케이준샐러드 고기도 꽉꽉, 채소도 꽉꽉 담겨있어요 치킨...",427,제과제빵떡케익제과점제과점업
180,파리바게뜨,제과제빵떡케익,제과점,제과점업,소사본동,37.479603,126.792995,소사본동%20파리바게뜨,https://m.place.naver.com/restaurant/19258625,경기 부천시 소사동로 82 센트럴프라자 105호,4.34,599,"/샐러드 가성비가 좋아요.. 케이준샐러드 고기도 꽉꽉, 채소도 꽉꽉 담겨있어요 치킨...",427,제과제빵떡케익제과점제과점업
247,파리바게뜨,제과제빵떡케익,제과점,제과점업,괴안동,37.479337,126.806668,괴안동%20파리바게뜨,https://m.place.naver.com/restaurant/19261450,경기 부천시 경인로 509 우남타워 1층 104호,4.35,1686,/직원분들 왕친절하세요👏👏👏😍/집근처에 있어서 자주 가요/호밀빵 최고!! 건강으로 ...,857,제과제빵떡케익제과점제과점업
